<h3 style="background-color:BurlyWood;text-align:center;color:Brown"> Importing certain tables and libraries</h3>

<h4 style="border:2px solid BurlyWood;"> This notebook gives a first look at the data. Before, we downloaded the tables from postgre-sql where a copy of clinical trials data has stored. Unfortunately The data stored as separate tables, therefore every variable/value has its own table and id.</h4>

<h4 style="border:2px solid Green;"> Here I import the tables, that I have found important for calculating success and failures</h4>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

<h4 style="border:2px solid Green;"> main table for studies. Has the informtion of phase, status, time frame...</h4>

In [ ]:
trials = pd.read_csv("../input/studiess/allstudies.csv")  
trials.head(5)

<h4 style="border:2px solid Green;"> I guess this table has the search words for trials, to make it easy to find...</h4>

In [ ]:
keywords = pd.read_csv("../input/keywords/keywords.csv") 
keywords.head(5) 

<h4 style="border:2px solid Green;"> below table has the other name of the intervention...</h4>

In [ ]:
int_oth_names = pd.read_csv("../input/intervetionothernames/intervention_other_names.csv") 
int_oth_names.head(5)  # this table can be helpful especially for sentiment analysis

<h4 style="border:2px solid Green;"> main table for interventions </h4>

In [ ]:
interventions = pd.read_csv("../input/interventions/interventions.csv")
interventions.head(5) 

<h4 style="border:2px solid Green;">table for withrawn studies </h4>

In [ ]:
drop_withdrawals = pd.read_csv("../input/dropwithdrawals/drop_withdrawals.csv")
drop_withdrawals.head(5) # this table has the reasons for failures or why a drug/intervention suspended/withdrawn

<h4 style="border:2px solid Green;"> below table has the same info with the "interventions" table, except for "group_type" column </h4>

In [ ]:
design_groups = pd.read_csv("../input/designgroups/design_groups.csv")
design_groups.head(5) 

<h4 style="border:2px solid Green;"> info about countries  </h4>

In [ ]:
countries = pd.read_csv("../input/countries/countries.csv")
countries.head(5) 
# "removed" column means, Removed location countries, Countries that appeared under listed location countries but were removed from the study record by the sponsor or investigator.

<h4 style="border:2px solid Green;"> main table for conditions/diseases </h4>

In [ ]:
conditions = pd.read_csv("../input/conditions/conditions.csv")
conditions.head(5) 

<h4 style="border:2px solid Green;"> mesh_term of interventions </h4>

In [ ]:
browse_interventions = pd.read_csv("../input/browseinterventions/browse_interventions.csv")
browse_interventions.head(5) 

<h4 style="border:2px solid Green;"> mesh_term of conditions </h4>

In [ ]:
browse_conditions = pd.read_csv("../input/browseconditions/browse_conditions.csv")
browse_conditions.head(5) 
browse_conditions["mesh_term"].nunique() # 4048

<h4 style="border:2px solid Green;">  mesh term headings,mesh tables do not have an NCT_id column, therefore only common column with other tables are id. </h4>

In [ ]:
mesh_head = pd.read_csv("../input/datassss/mesh_headings.csv")
mesh_head.head(5) 

In [ ]:
mesh_terms = pd.read_csv("../input/datassss/mesh_terms.csv")
mesh_terms.head(5)

<h4 style="border:2px solid Green;"> sponsors </h4>

In [ ]:
sponsors = pd.read_csv("../input/sponsorparty/sponsors.csv")
sponsors.head(5) # sponsors

<h4 style="border:2px solid Green;"> responsible party of the trial </h4>

In [ ]:
res_party = pd.read_csv("../input/sponsorparty/responsible_parties.csv")
res_party.head(5) 

<h4 style="border:3px solid IndianRed ; background-color:Ivory;text-align:center;color:LightSeaGreen"> Checking DataFrame </h4>

Tables of interest;

* trials 
* keywords
* int_oth_names
* interventions
* drop_withdrawals
* design_groups
* countries
* conditions
* browse_interventions
* browse_conditions
* mesh_head
* mesh_terms
* sponsors
* res_party

In [ ]:
def data_check(dataframe):
    print("-------- column names ------------")
    print(dataframe.columns)
    print("-------- shape before merge ---------")
    print(dataframe.shape)
    print("--------- null values -------")
    print(dataframe.isnull().sum())
    print("--------- any duplicates in NCT_İD ------")
    print(dataframe.duplicated(subset=["nct_id"], keep='first').sum())
    
data_check(trials)

In [ ]:
# bringing important columns instead all dataframe for faster results
some_columns = ['nct_id','start_month_year', 'start_date_type',
       'start_date', 'completion_month_year', 'completion_date', 'study_type', 'acronym','brief_title', 'official_title',
       'overall_status', 'last_known_status', 'phase', 'enrollment',
       'enrollment_type','why_stopped', "number_of_arms","source" ]
trials__ = trials[some_columns]
trials__.shape # (383720, 18)
trials__.head()

<h4 style="border:3px solid IndianRed ; background-color:Ivory;text-align:center;color:LightSeaGreen"> last 15 years trials </h4>

In [ ]:
trials__['completion_date'] = pd.to_datetime(trials__['completion_date'])
trials_new = trials__.loc[(trials__['completion_date'] >= 'December 30 2004')
                     & (trials__['completion_date'] < 'August 1 2021')]
trials_new.shape # (276975, 16)
min(trials_new["completion_date"]) # Timestamp('2004-12-31 00:00:00')
max(trials_new["completion_date"]) # Timestamp('2021-07-31 00:00:00')

<h4 style="border:3px solid IndianRed ; background-color:Ivory;text-align:center;color:LightSeaGreen"> Basic Observations </h4>

<h4 style="background-color:LightCyan;color:LightSeaGreen"> What is the number of complited studies over years? </h4>

In [ ]:
pd.DatetimeIndex(trials_new['completion_date']).year

import plotly.graph_objs as go

fig = go.Figure(go.Bar(
    x= trials_new.groupby(pd.DatetimeIndex(trials_new['completion_date']).year).agg('count')["nct_id"].sort_values(ascending=False).index,
    y= trials_new.groupby(pd.DatetimeIndex(trials_new['completion_date']).year).agg('count')["nct_id"].sort_values(ascending=False).values, 
    textposition='outside' ))
fig.update_layout(title='Studies Per Year')
fig.show()

<h4 style="background-color:LightCyan;color:LightSeaGreen"> What is the general status of trials? </h4>

In [ ]:
from plotly import __version__
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
trials_new['overall_status'].value_counts().sort_values(ascending=True).iplot(kind='barh',xTitle='Count',color='green')

<h4 style="background-color:LightCyan;color:LightSeaGreen"> What is the most popular study type? </h4>

In [ ]:
labels = trials_new['study_type'].value_counts().index
values = trials_new['study_type'].value_counts().values
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial')])
fig.show()

<h4 style="background-color:LightCyan;color:LightSeaGreen"> How large are Interventional Trials? </h4>

In [ ]:
interventional_studies = trials_new[trials_new['study_type']=='Interventional']
# Convert to numeric
## interventional_studies['enrollment'] = interventional_studies['enrollment'].astype(int)
# Remove the trials with recruitment status withdrawn and terminated
enrollment = interventional_studies.loc[
 (interventional_studies['overall_status'] != 'Withdrawn') & (interventional_studies['overall_status'] != 'Terminated')]
bins = [-1, 20, 40, 60, 100, 200, 400, 600, 1000]
group_names = ['< 20', '21-40', '41-60', '61-100', '101-200', '201-400', '401-600', '>600']
categories = pd.cut(enrollment['enrollment'], bins, labels=group_names)
# Add categories as column in dataframe
enrollment['Category'] = categories
# View value counts
enrollment_counts = enrollment['Category'].value_counts().sort_index(ascending=True)
enrollment_counts.iplot(kind='bar', title='Size of Interventional Trials',  colors='LightGreen')

<h4 style="background-color:LightCyan;color:LightSeaGreen"> What are the percentage of the phases? </h4>

In [ ]:
labels = trials_new['phase'].value_counts().drop('Not Applicable').index
values = trials_new['phase'].value_counts().drop('Not Applicable').values
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label',
                             insidetextorientation='radial')])
fig.show()

<h4 style="border:3px solid IndianRed ; background-color:Ivory;text-align:center;color:LightSeaGreen"> Merging tables and determining the final dataframe </h4>

In [ ]:
trials_new.duplicated(subset=["nct_id"], keep='first').sum()

In [ ]:
interventions.duplicated(subset=["nct_id"], keep='first').sum() # 312 718
interventions_no_duplicates = interventions.drop_duplicates(subset=['nct_id'], keep=False)
interventions_no_duplicates.shape # (159496, 5)
interventions_no_duplicates.columns # ['id', 'nct_id', 'intervention_type', 'name', 'description']
interventions_last_df = interventions_no_duplicates.rename(columns={'id': 'id_interventions', 'name': 'intervention_name','description': 'intervention_description' })

In [ ]:
int_oth_names.head(3) # the other name of the intervention can be usefull 
int_oth_names.duplicated(subset=["nct_id"], keep='first').sum() # 220469
int_oth_names_no_duplicates = int_oth_names.drop_duplicates(subset=['nct_id'], keep=False)
int_oth_names_no_duplicates.shape # (51903, 4)

In [ ]:
keywords.sample(5) # seems like this consists of resarch keywords for search engine
keywords.duplicated(subset=["nct_id"], keep='first').sum() # 220469
keywords_no_duplicates = keywords.drop_duplicates(subset=['nct_id'], keep=False)
keywords_no_duplicates.shape # (38655, 4)

In [ ]:
design_groups.head() # has the same information with interventions table
design_groups.duplicated(subset=["nct_id"], keep='first').sum() # 355 989
design_groups_no_duplicates = design_groups.drop_duplicates(subset=['nct_id'], keep=False)
design_groups_no_duplicates.shape # (95851, 5)

In [ ]:
drop_withdrawals.head() # this has the withdrawal studies reasons 

In [ ]:
countries.head()
countries.duplicated(subset=["nct_id"], keep='first').sum() # 195 885
countries_no_duplicates = countries.drop_duplicates(subset=['nct_id'], keep=False)
countries_no_duplicates.shape # (310944, 4)

In [ ]:
browse_interventions.head(10) # interventions browse mesh-terms
browse_interventions.duplicated(subset=["nct_id"], keep='first').sum() # 126 300
browse_interventions_no_duplicates = browse_interventions.drop_duplicates(subset=['nct_id'], keep=False)
browse_interventions_no_duplicates.shape # (74467, 4)

In [ ]:
browse_conditions.head(10) # conditions browse mesh terms
browse_conditions.duplicated(subset=["nct_id"], keep='first').sum() # 329 883
browse_conditions_no_duplicates = browse_conditions.drop_duplicates(subset=['nct_id'], keep=False)
browse_conditions_no_duplicates.shape # (143104, 4)

In [ ]:
conditions.head(8) # name of the conditions 
conditions.duplicated(subset=["nct_id"], keep='first').sum() # 268 859
conditions_no_duplicates = conditions.drop_duplicates(subset=['nct_id'], keep=False)
conditions_no_duplicates.shape # (257 627, 4)
conditions_no_duplicates.columns
conditions_last_df = conditions_no_duplicates.rename(columns={'id': 'id_conditions', 'name': 'condition_name','downcase_name': 'condition_downcase_name' })

In [ ]:
# mesh terms do not have nct_id
# mesh_head.head(5) # mesh term headings 
# mesh_head.duplicated(subset=["nct_id"], keep='first').sum() # 268 859
# conditions_no_duplicates = conditions.drop_duplicates(subset=['nct_id'], keep=False)
# conditions_no_duplicates.shape # (257 627, 4)

In [ ]:
# mesh_terms.head(5)
# conditions.duplicated(subset=["nct_id"], keep='first').sum() # 268 859
# conditions_no_duplicates = conditions.drop_duplicates(subset=['nct_id'], keep=False)
# conditions_no_duplicates.shape # (257 627, 4)

In [ ]:
sponsors.head()
sponsors.duplicated(subset=["nct_id"], keep='first').sum() # 230 392
sponsors_no_duplicates = sponsors.drop_duplicates(subset=['nct_id'], keep=False)
sponsors_no_duplicates.shape # (255 921, 5)
sponsors_no_duplicates.columns # ['id', 'nct_id', 'agency_class', 'lead_or_collaborator', 'name']
sponsors_last_df = sponsors_no_duplicates.rename(columns={'id': 'id_sponsor', 'agency_class': 'sponsor_agency_class','lead_or_collaborator': 'sponsor_lead_or_collaborator',
                                                             'name': 'sponsor_name' })

In [ ]:
res_party.head(5)
res_party.duplicated(subset=["nct_id"], keep='first').sum() # 0 duplicates

res_party.columns # ['id', 'nct_id', 'responsible_party_type', 'name', 'title','organization', 'affiliation']
res_party_last_df = res_party.rename(columns={'id': 'id_responsible_party', 'name': 'responsible_party_name','title': 'responsible_party_title',
                                                             'organization': 'responsible_party_organization', 'affiliation': 'responsible_party_affiliation' })

In [ ]:
outcome_analysis_raw = pd.read_csv("../input/outcome-analyses/outcome_analyses.csv")
outcome_analysis_raw.columns

In [ ]:
outcome_analysis_raw.head(10)
cols_in_ourcome_analy = ['nct_id', 'p_value', 'p_value_description', 'method']
outcome_analysis= outcome_analysis_raw[cols_in_ourcome_analy]
outcome_analysis.shape # (213198, 4)
outcome_analysis.duplicated(subset=["nct_id"], keep='first').sum() # 195674
outcome_analysis_no_duplicates = outcome_analysis.drop_duplicates(subset=['nct_id'], keep=False)
outcome_analysis_no_duplicates.shape 

In [ ]:
merge_1 = pd.merge(trials_new, conditions_last_df, on="nct_id")
merge_1.shape  # (192398, 21)
merge_2 = pd.merge(merge_1, interventions_last_df, on="nct_id")
merge_2.shape # (78823, 25)
merge_2.columns
merge_3 = pd.merge(merge_2, sponsors_last_df, on="nct_id")
merge_3.shape # (55582, 29)
merge_3.columns
merge_4 = pd.merge(merge_3, res_party_last_df, on="nct_id")
merge_4.shape # (52578, 35)
merge_4.columns
# merge_5 = pd.merge(merge_4, outcome_analysis, on="nct_id")
# merge_5.shape # (13604, 38) # decrease the shape of the data tremendously

In [ ]:
merge_4.duplicated(subset=["nct_id"], keep='first').sum() # 0 duplicates
dataframe = merge_4.copy()
dataframe.sample(3)

In [ ]:
dataframe.to_csv("./dataframe.csv")

İntervention Features:

* intervention type
* intervention name
* intervention other name
* acronym

trial features:

* nct_id
* country
* condition
* Overall Status
* enrollment
* Start Date
* Completion Date
* Responsible Party Type
* Sponsor lead or collabarator
* Sponsor Agency Class

Features outside

* Biomarker
* Scientist Sentiment from Twitter

<h4 style="background-color:LightGrey;color:Maroon"> What we have in dataframe ? </h4>

<h4 style="background-color:LightGrey;color:LightSlateGray"> DataFrame consists of: </h4>
 
<h5 style="background-color:Snow;color:SteelBlue">
    
last 15 years trials, 
    
interventions; names, types, description and acronym
    
conditions; names, other names, downcase names
    
trials; enrollment number, enrollment type, study type, study brief title, study official title, completion and start dates, phase, last known status, if stopped/why stopped, sponsor, agency class, responsible party organization information</h5>